# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

Below we are adding the dependencies.


In [2]:
import logging
import os
import csv
import joblib
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
import pkg_resources
import xgboost


from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AGKHK3JXL to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-139938
aml-quickstarts-139938
southcentralus
9a7511b8-150f-4a58-8528-3e7d50216c31


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.





In [4]:
experiment_name = 'Experiment_on_Heart-Failure-Prediction'
project_folder = './capstone-classification-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
Experiment_on_Heart-Failure-Prediction,quick-starts-ws-139938,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
We will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for our AutoML run. In this tutorial, we get the default `AmlCompute` as our training compute resource.


In [5]:
amlcompute_cluster_name = "aml-compute1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset I used is heart_failure_clinical_records_dataset.csv(https://www.kaggle.com/andrewmvd/heart-failure-clinical-data). It describes some recorded health indicators metrics. The data have almost 300 rows of these indicators recorded from patients.

I am using this data in order to predict the DEATH_EVENT i.e. whether or not the patient deceased during the follow-up period (boolean). The features of the data are the following:

age : The age of the patient.

anaemia : Decrease of red blood cells or hemoglobin (boolean).

creatinine_phosphokinase : Decrease of red blood cells or hemoglobin (boolean).

diabetes : If the patient has diabetes (boolean).

ejection_fraction : Percentage of blood leaving the heart at each contraction (percentage).

high_blood_pressure : If the patient has hypertension (boolean).

platelets : Platelets in the blood (kiloplatelets/mL).

serum_creatinine : Level of serum creatinine in the blood (mg/dL).

serum_sodium : Level of serum sodium in the blood (mEq/L).

sex : Woman or man (binary).

smoking : If the patient smokes or not (boolean).

time : Follow-up period (days).

DEATH_EVENT : If the patient deceased during the follow-up period (boolean).


In [6]:
found = False
key = "Heart-Failure"
description_text = "Heart Failure csv for prediction for the Capstone Project."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/michav1510/Capstone-Project-Azure-Machine-Learning-Engineer/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,sex,time
count,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.000000
mean,60.833893,581.839465,38.083612,263358.029264,1.39388,136.625418,0.648829,130.260870
std,11.894809,970.287881,11.834841,97804.236869,1.03451,4.412477,0.478136,77.614208
min,40.000000,23.000000,14.000000,25100.000000,0.50000,113.000000,0.000000,4.000000
25%,51.000000,116.500000,30.000000,212500.000000,0.90000,134.000000,0.000000,73.000000
50%,60.000000,250.000000,38.000000,262000.000000,1.10000,137.000000,1.000000,115.000000
75%,70.000000,582.000000,45.000000,303500.000000,1.40000,140.000000,1.000000,203.000000
max,95.000000,7861.000000,80.000000,850000.000000,9.40000,148.000000,1.000000,285.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
#automl_settings = {
#    "experiment_timeout_minutes": 30,
#    "n_cross_validations": 2,
#    "enable_early_stopping": True, 
#    "max_concurrent_iterations": 5,
#    "primary_metric" : 'accuracy'
#}

automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

#automl_config = AutoMLConfig(
#                             task = "classification",
#                             training_data=dataset,
#                             label_column_name="DEATH_EVENT",   
#                             path = project_folder,
#                             featurization= 'auto',
#                             debug_log = "automl_errors.log",
#                             **automl_settings
#                            )

automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            blocked_models = ['XGBoostClassifier'],
                            **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)
remote_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on aml-compute1 with default configuration
Running on remote compute: aml-compute1
Parent Run ID: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation

{'runId': 'AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926',
 'target': 'aml-compute1',
 'status': 'Completed',
 'startTimeUtc': '2021-03-06T14:20:01.929129Z',
 'endTimeUtc': '2021-03-06T14:35:24.744035Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'aml-compute1',
  'AMLSettingsJsonString': '{"path":null,"name":"Experiment_on_Heart-Failure-Prediction","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-139938","workspace_name":"quick-starts-ws-139938","region":"southcentralus","compute_target":"aml-compute1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":null,"y_max

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In the cell below, we use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
for children_run in remote_run.get_children():
    print('------')
    print(children_run)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

------
Run(Experiment: Experiment_on_Heart-Failure-Prediction,
Id: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_36,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: Experiment_on_Heart-Failure-Prediction,
Id: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_37,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: Experiment_on_Heart-Failure-Prediction,
Id: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_35,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: Experiment_on_Heart-Failure-Prediction,
Id: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_34,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: Experiment_on_Heart-Failure-Prediction,
Id: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_32,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: Experiment_on_Heart-Failure-Prediction,
Id: AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_33,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: Experiment_on_Heart-Failur

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-core, training version:1.23.0, current version:1.22.0
Package:azureml-dataprep, training version:2.10.1, current version:2.9.1
Package:azureml-dataprep-native, training version:30.0.0, current version:29.0.0
Package:azureml-dataprep-rslex, training version:1.8.0, current version:1.7.0
Package:azureml-dataset-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-defaults, training version:1.23.0, current version:1.22.0
Package:azureml-interpret, training version:1.23.0, current version:1.22.0
Package:azureml-mlflow, training version:1.23.0, current version:1.22.0
Package:azureml-pipeline-core, training version:1.23.0, current version:1.22.0
Package:azureml-telemetry, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-client, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-runtime, training version:1.23.0, current versio

Below we have the best run

In [12]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Experiment_on_Heart-Failure-Prediction,AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


Below we will see the best run metrics

In [13]:
best_run.get_metrics()

{'recall_score_macro': 0.8067028897086537,
 'recall_score_micro': 0.8595525727069351,
 'recall_score_weighted': 0.8595525727069351,
 'precision_score_micro': 0.8595525727069351,
 'average_precision_score_macro': 0.884023494734937,
 'norm_macro_recall': 0.6134057794173073,
 'average_precision_score_micro': 0.9045990663819037,
 'precision_score_weighted': 0.8647516519285114,
 'f1_score_micro': 0.8595525727069352,
 'f1_score_macro': 0.8256222943722944,
 'precision_score_macro': 0.8690786415973855,
 'f1_score_weighted': 0.8529733650018885,
 'accuracy': 0.8595525727069351,
 'AUC_micro': 0.9057011666141166,
 'weighted_accuracy': 0.9005775096728179,
 'log_loss': 0.4145328260442895,
 'AUC_weighted': 0.8985222275132931,
 'AUC_macro': 0.898522227513293,
 'average_precision_score_weighted': 0.9058096781656662,
 'balanced_accuracy': 0.8067028897086537,
 'matthews_correlation': 0.6719313047639512,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_36

Below we will see the best run details

In [14]:
best_run.get_details()

{'runId': 'AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_36',
 'target': 'aml-compute1',
 'status': 'Completed',
 'startTimeUtc': '2021-03-06T14:34:10.54728Z',
 'endTimeUtc': '2021-03-06T14:35:13.250305Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Experiment_on_Heart-Failure-Prediction\',\'compute_target\':\'aml-compute1\',\'subscription_id\':\'9a7511b8-150f-4a58-8528-3e7d50216c31\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_36","experiment_name":"Experiment_on_Heart-Failure-Prediction","workspace_name":"quick-starts-ws-139

Below we will see the properties of best run

In [15]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Experiment_on_Heart-Failure-Prediction\',\'compute_target\':\'aml-compute1\',\'subscription_id\':\'9a7511b8-150f-4a58-8528-3e7d50216c31\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_b0c1a3bd-1c33-4019-a058-a8916bc05926_36","experiment_name":"Experiment_on_Heart-Failure-Prediction","workspace_name":"quick-starts-ws-139938","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group_name":"aml-quickstarts-139938"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '36',
 '_aml_system_scenario_identifica

Below we save the best model

In [16]:
best_run.register_model(model_name='best_run_capston_automl', model_path='./outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-139938', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-139938'), name=best_run_capston_automl, id=best_run_capston_automl:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
model = remote_run.register_model(model_name = 'best_run_automl.pkl')

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
%run endpoint.py

{"result": [true, true]}
++++++++++++++++++++++++++++++
Expected result: [true, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
print(service.get_logs())

2021-03-06T14:41:04,318461800+00:00 - nginx/run 
2021-03-06T14:41:04,320127400+00:00 - iot-server/run 
2021-03-06T14:41:04,320885900+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-03-06T14:41:04,345541700+00:00 - rsyslog/run 
rsyslogd

In [21]:
service.delete()